In [1]:
import numpy as np
import math
import random
import matplotlib.pyplot as plt
import pandas as pd
import copy
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score
from scipy import stats
from sklearn.metrics import confusion_matrix
from sklearn.metrics import jaccard_score
from sklearn.metrics import f1_score
from sklearn.feature_selection import mutual_info_classif
from sklearn.neighbors import KNeighborsClassifier 

In [2]:
#加载数据 预处理
data = pd.read_csv("../data/gene_version2020/generate/ExperimentalDatasets/MM/MM-BP.csv",index_col=0)
label = data["longevity influence"]
#删除无用列
data = data.drop(["longevity influence"],axis=1)

#初始化基因本体的DAG
DAG = pd.read_csv("../data/gene_version2020/generate/GOPath/MM/MM-BP.csv",index_col=0)

threshold = 0.999
AncDict = {}
DecDict = {}
relativeList = []
sparseList = []
sensitivity = []
specificity = []
F1 = []
AUC = []
featureNum = []
pathList = []

In [5]:
#过滤低纬度特征
for c in data.columns:
    data[c] = data[c].astype('int')
    if(data[c].sum() < 5):
        data.drop(c,axis=1,inplace=True)
DAG = DAG.loc[data.columns,data.columns]

#计算相关性
IGWeight = mutual_info_classif(data,label,discrete_features=True)
IGWeight = pd.Series(IGWeight,index=DAG.columns)

In [14]:
sparseList = []
#根据稀疏过滤阈值标记稀疏特征
for c in data.columns:
    data[c] = data[c].astype('int')
    if(abs(0.5-data[c].mean()) > 0.49):
        print(abs(0.5-data[c].mean()))
        #data.drop(c,axis=1,inplace=True)
        sparseList.append(c)
#DAG = DAG.loc[data.columns,data.columns]

0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234


0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234


0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234
0.49230769230769234


In [4]:
#计算相关性
data["label"] = label
Relevance1 = pd.Series(0.0,index=DAG.columns)
for name in DAG.columns:
    temp = data[data[name] == 1]
    prob = temp['label'].mean()
    Relevance1[name] = (prob-0.5)*(prob-0.5)+(0.5-prob)*(0.5-prob)
    
Relevance0 = pd.Series(0.0,index=DAG.columns)
for name in DAG.columns:
    temp = data[data[name] == 0]
    prob = temp['label'].mean()
    Relevance0[name] = (prob-0.5)*(prob-0.5)+(0.5-prob)*(0.5-prob)  
data = data.drop(["label"],axis=1)

#计算所有节点的祖先节点
for name in DAG.columns:
    global relativeList
    relativeList = []
    getAncestors(name)
    AncDict[name] = relativeList
#计算所有节点的孩子节点
for name in DAG.columns:
    global relativeList
    relativeList = []
    getDecestors(name)
    DecDict[name] = relativeList

In [3]:
def SHSEL(trainData):
    global pathList
    #找到所有叶子结点
    leafNodes = []
    dropList = []
    returnList = []
    for go in DAG.columns:
        if(DAG.loc[go,:].sum() == 0):
            leafNodes.append(go)
    #step1
    for leaf in leafNodes:
        for node in DAG.columns:
            if(DAG.loc[node,leaf] == 1):
                if((1-abs(IGWeight[node]-IGWeight[leaf])) >= threshold):
                    dropList.append(leaf)
                leafNodes.append(node)
    selectFeatures = list(set(DAG.columns)-set(dropList))
    print(len(selectFeatures))
    #step2
    leafNodes = []
    for go in DAG.columns:
        if(DAG.loc[go,:].sum() == 0):
            leafNodes.append(go)
    for leaf in leafNodes:
        pathList = []
        getPathsToRoot([],leaf)
        for path in pathList:
            path = list(filter(lambda x:x in selectFeatures,path))
            pathIG = IGWeight[path]
            highIG = pathIG[pathIG >= pathIG.mean()]
            for index in highIG.index:
                returnList.append(index)
    return list(set(returnList))

def GM_score(x,y): #计算GM 传入numpy
    cm1 = confusion_matrix(x,y)
    sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    specificity1 = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    return sensitivity1,specificity1

def getPathsToRoot(path,name):#传入list 方便递归
    global pathList
    flag = 0
    path.append(name)
    for anc in DAG.columns:
        if(DAG.loc[anc,name] == 1):
            getPathsToRoot(copy.copy(path),anc)
            flag = 1
    if(flag == 0):
        pathList.append(path)
        
def RPV(x):
    dropList = []
    for name in DAG.columns:
        if(x[name] == 1):
            #遍历当前节点的所有祖先节点
            for anc in AncDict[name]:
                if(Relevance1[anc] < Relevance1[name]):
                    dropList.append(anc)
        else:
            dropList.append(name)
    return list(set(DAG.columns)-set(dropList))

def MR(x):
    global pathList
    dropSet = set()
    for name in DAG.columns:
        pathList = []
        if(x[name] == 1):#找到所有至根节点路径
            getPathsToRoot([],name)
            for path in pathList:
                #保留相关性最高特征，其它特征加入dropSet
                dropSet.update(Relevance1[path].sort_values(ascending=False).index[1:])
        else:
            dropSet.add(name)
#             getPathsToLeft([],name)
#             for path in pathList:
#                 #保留相关性最高特征，其它特征加入dropSet
#                 dropSet.update(Relevance0[path].sort_values(ascending=False).index[1:])
        #print(pathList)
    return list(set(DAG.columns)-dropSet)

def myknn(train,label,x_test):
    predictList = []
    for j in range(x_test.shape[0]):
        test = x_test.iloc[j,:]
        maxJcd = 0
        prediction = 0
        for i in range(train.shape[0]):
            temp = jaccard_score(train.iloc[i,:],test)
            if(temp > maxJcd):
                prediction = label[i]
                maxJcd = temp
        predictList.append(prediction)
    return predictList

def getAncestors(name):
    global relativeList
    for relative in DAG.columns:
        if(DAG.loc[relative,name] == 1):
            relativeList.append(relative)
            getAncestors(relative)
def getDecestors(name):
    global relativeList
    for relative in DAG.columns:
        if(DAG.loc[name,relative] == 1):
            relativeList.append(relative)
            getDecestors(relative)
def getPathsToRoot(path,name):#传入list 方便递归
    global pathList
    flag = 0
    path.append(name)
    for anc in DAG.columns:
        if(DAG.loc[anc,name] == 1):
            getPathsToRoot(copy.copy(path),anc)
            flag = 1
    if(flag == 0):
        pathList.append(path)
def getPathsToLeft(path,name):#传入list 方便递归
    global pathList
    flag = 0
    path.append(name)
    for dec in DAG.columns:
        if(DAG.loc[name,dec] == 1):
            flag = 1
            getPathsToLeft(copy.copy(path),dec)
    if(flag == 0):
        pathList.append(path)

In [32]:
#10折交叉验证
kf = KFold(n_splits=10,shuffle=True)
for train_index ,test_index in kf.split(data):
    trainData = data.iloc[train_index,:]
    testData = data.iloc[test_index,:]
    Y_train = label.values[train_index] #用于训练模型
    Y_test = label.values[test_index]#用于交叉验证
    predicetList = []
    
    Features = SHSEL(trainData)#第一层特征选择 
    for i in range(testData.shape[0]):  #对每一个样本进行特征选择和预测
        
        selectFeatures = RPV(testData.iloc[i,:]) #RPV特征选择
        #selectFeatures = MR(testData.iloc[i,:]) #MR特征选择
        
        print(len(Features))
        print(len(selectFeatures))
        
        selectFeatures = list(set(Features) & set(selectFeatures))
        if(len(selectFeatures) == 0):
            selectFeatures = DAG.columns
        X_train = trainData.loc[:,selectFeatures]
        X_test = testData.loc[:,selectFeatures].iloc[i,:]
        
        gnb = GaussianNB()
        predicetList.append(gnb.fit(X_train, Y_train).predict(X_test.values.reshape(1,len(selectFeatures)))[0])#把当前样本预测结果加入List
        #knn=KNeighborsClassifier()
        #knn.fit(X_train,Y_train)
        #predicetList.append(knn.predict(X_test.values.reshape(1,len(selectFeatures)))[0])
    print(predicetList)
    print(Y_test)
    sensi,speci= GM_score(np.array(predicetList),Y_test)
    sensitivity.append(sensi)
    specificity.append(speci)
    F1.append(f1_score(np.array(predicetList),Y_test))
    try:
        AUC.append(roc_auc_score(np.array(predicetList),Y_test))
    except ValueError:
        pass

116
92
12
92
5
92
7
92
5
92
2
92
1
92
7
92
3
92
16
92
21
92
7
92
5
92
1
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[1 1 0 1 1 1 1 1 1 0 1 1 1]


<ipython-input-3-18b1d8e33a01>:37: RuntimeWarning: invalid value encountered in longlong_scalars
  sensitivity1 = cm1[0,0]/(cm1[0,0]+cm1[0,1])


116
92
16
92
8
92
1
92
5
92
3
92
10
92
4
92
5
92
0
92
14
92
3
92
3
92
3
[1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1]
[1 0 0 0 1 1 0 1 0 1 1 0 1]
116
92
6
92
3
92
5
92
0
92
1
92
1
92
4
92
4
92
5
92
6
92
6
92
5
92
4
[1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0]
[1 1 1 1 1 0 1 1 0 0 1 0 1]
116
92
2
92
9
92
5
92
8
92
21
92
2
92
30
92
0
92
1
92
0
92
15
92
4
92
5
[1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0]
[1 1 1 1 0 1 1 0 1 0 1 1 1]
116
92
23
92
2
92
3
92
3
92
11
92
10
92
2
92
9
92
1
92
9
92
14
92
1
92
8
[1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0]
[1 1 0 1 1 1 1 0 1 1 0 1 1]
116
92
3
92
1
92
1
92
4
92
7
92
5
92
7
92
27
92
11
92
7
92
12
92
1
92
4
[0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0]
[0 1 1 1 0 0 1 1 1 0 1 0 0]
116
92
4
92
10
92
9
92
1
92
14
92
0
92
8
92
10
92
0
92
8
92
16
92
2
92
2
[1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0]
[1 1 1 1 0 1 1 0 0 1 0 0 1]
116
92
5
92
7
92
3
92
2
92
6
92
12
92
4
92
19
92
1
92
13
92
8
92
8
92
7
[0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0]
[0 0 1 1 1 1 1 1 1 1 0 1 1]
116
92
7
92
4
92
17
9

In [33]:
a = np.nanmean(sensitivity)
b = np.nanmean(specificity)
print(a)
print(b)
print("GM")
print(math.sqrt(a*b))
print("f1")
print(np.nanmean(F1))
print("AUC")
print(np.nanmean(AUC))

0.473831324550749
0.7755273099915958
GM
0.6061923230449204
f1
0.6587879197808432
AUC
0.624425264982819


In [26]:
Relevance1+Relevance0

GO:0005515    0.110857
GO:0042802    0.090555
GO:0042803    0.114788
GO:0050661    0.542439
GO:0003779    0.068672
                ...   
GO:0097371    0.542439
GO:0004407    0.542439
GO:0019213    0.542439
GO:0070403    0.542439
GO:0043022    0.107667
Length: 100, dtype: float64